## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
sample_no = 3
data_dir = f"../data/Polypharmacy/sample{sample_no}/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_12.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_22.pkl",'rb'))
# V1 = pkl.load(open(data_dir+"X_26.pkl",'rb'))
# W1 = pkl.load(open(data_dir+"X_53.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_11_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_11_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_11_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_11_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"V1":V1,"R":R_temp_dict}

Loading data from data_dir:  ../data/Polypharmacy/sample3/


In [6]:
print("U1.shape: ",U1.shape)
print("V1.shape: ",V1.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (645, 837)
V1.shape:  (837, 837)
R.shape:  (645, 645)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X2"],\
    "e2":["X2","X3"]}
    #"e6":["X4"]}

In [8]:
X_data = {
    "X1":{"1":data_dict['R'][1]["Rtrain"]},\
    "X2":{"1":U1},\
    "X3":V1}

In [9]:
X_meta = {
    "X1":["e1","e1"],\
    "X2":["e1","e2"],\
    "X3":["e2","e2"]}
    #"X5":["e5","e3"]}

In [10]:
# Rtest_triplets1 = [[1,1,1],[2,2,0]]
# Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]
Rtest_triplets1 = [[241, 208, 0], [111, 37, 1]]
Rtest_triplets2 = [[1, 159, 1], [0, 0, 0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  2
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  2
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X2']
X_id:  X1
X[X_id].shape:  (645, 645)
X_id:  X2
X[X_id].shape:  (645, 837)
C_dict[e].shape:  torch.Size([645, 1482])
---
e_id:  e2
X_id_list:  ['X2', 'X3']
X_id:  X2
X[X_id].shape:  (645, 837)
X_id:  X3
X[X_id].shape:  (837, 837)
C_dict[e].shape:  torch.Size([837, 1

#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [19]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2'])

In [20]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3'])

In [21]:
dcmf_model.out_dict_info

{'params': {'learning_rate': 0.001,
  'weight_decay': 0.05,
  'convg_thres': 0.1,
  'max_epochs': 5,
  'is_pretrain': True,
  'pretrain_thres': 0.1,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'k': 100,
  'kf': 0.5,
  'e_actf': 'tanh',
  'd_actf': 'tanh',
  'is_linear_last_enc_layer': False,
  'is_linear_last_dec_layer': False},
 'num_val_sets': 1,
 'loss_all_folds': {'1': [0.24306784570217133,
   0.05822769645601511,
   0.28170935809612274,
   0.05353531055152416,
   0.07973881997168064]},
 'loss_all_folds_avg_tuple': [0.24306784570217133,
  0.05822769645601511,
  0.28170935809612274,
  0.05353531055152416,
  0.07973881997168064],
 'loss_all_folds_avg_sum': 0.716279030777514,
 'val_metric': 'auc',
 'val_perf_all_folds': {'1': {'X1': 1.0, 'X2': 0.0}},
 'val_perf_all_folds_avg': {'X1': 1.0, 'X2': 0.0},
 'val_perf_all_folds_total': {'1': 1.0},
 'val_perf_all_folds_total_avg': 1.0,
 'E': 2,
 'M': 3}

In [22]:
drug_emb = list(dcmf_model.out_dict_U['1'].values())[0].detach().numpy()
protein_emb = list(dcmf_model.out_dict_U['1'].values())[1].detach().numpy()

In [23]:
import pandas as pd

In [24]:
drug_emb_df = pd.DataFrame(drug_emb)
drug_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.032725,-0.066563,-0.005550,-0.021116,-0.004270,-0.006089,-0.073543,-0.007728,0.033146,0.049541,...,0.008882,0.046231,0.045813,-0.053337,0.062458,0.049581,0.034330,-0.000227,-0.045746,0.034527
1,-0.046697,-0.043965,0.017486,-0.019335,-0.002567,-0.024133,-0.067445,0.012660,0.035056,0.035366,...,0.018196,0.041177,0.039334,-0.044318,0.045463,0.025545,0.049202,0.000823,-0.038362,0.019681
2,-0.021999,-0.132681,-0.008762,-0.042748,-0.019589,-0.075033,-0.132914,0.005916,0.003201,0.049293,...,-0.015176,0.088349,0.074305,-0.060316,0.136529,0.038422,-0.004120,-0.041907,-0.089914,-0.001653
3,-0.058334,-0.032472,0.008695,-0.011285,-0.004759,-0.007067,-0.046322,0.025255,0.045424,0.050698,...,0.007130,0.038114,0.035892,-0.047891,0.040576,0.048087,0.053413,0.015304,-0.030066,0.023187
4,-0.029307,-0.066059,0.003879,-0.029705,-0.008513,-0.051460,-0.089139,-0.002922,0.013436,0.054816,...,0.013478,0.049914,0.044104,-0.035458,0.086094,0.047663,0.037057,-0.030735,-0.054160,0.024727


In [25]:
drug_emb_df.shape

(645, 100)

In [26]:
prot_emb_df = pd.DataFrame(protein_emb)
prot_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.034884,-0.041900,0.014441,-0.028866,0.024075,-0.050213,-0.055515,-0.003073,-0.039138,-0.050598,...,-0.000543,-0.001204,0.024938,0.000515,-0.032049,-0.025672,0.014274,0.061299,-0.048068,0.007866
1,-0.028947,-0.044060,0.015945,-0.026450,0.020862,-0.043119,-0.061352,-0.004587,-0.039323,-0.054134,...,0.000107,-0.000914,0.025148,0.002490,-0.033825,-0.028532,0.012723,0.056688,-0.049348,0.008641
2,-0.022945,-0.039050,0.018115,-0.025355,0.028407,-0.050784,-0.041365,-0.009819,-0.028312,-0.043993,...,-0.004336,0.004620,0.028203,-0.004753,-0.031010,-0.035759,0.019421,0.060457,-0.053165,-0.002401
3,-0.032037,-0.043101,0.017195,-0.026225,0.030205,-0.044417,-0.054503,-0.003304,-0.040402,-0.049562,...,0.000485,-0.002409,0.024773,0.000728,-0.034464,-0.028689,0.014303,0.056698,-0.049923,0.006563
4,-0.032398,-0.045110,0.014065,-0.025320,0.024967,-0.045400,-0.057338,-0.002656,-0.041043,-0.051630,...,-0.001580,-0.004056,0.024718,-0.001951,-0.034214,-0.026462,0.018331,0.058531,-0.045523,0.009722


In [27]:
prot_emb_df.shape

(837, 100)

In [28]:
emb_df = pd.concat([drug_emb_df, prot_emb_df], ignore_index = True, axis = 0)
emb_df.shape

(1482, 100)

In [29]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.032725,-0.066563,-0.005550,-0.021116,-0.004270,-0.006089,-0.073543,-0.007728,0.033146,0.049541,...,0.008882,0.046231,0.045813,-0.053337,0.062458,0.049581,0.034330,-0.000227,-0.045746,0.034527
1,-0.046697,-0.043965,0.017486,-0.019335,-0.002567,-0.024133,-0.067445,0.012660,0.035056,0.035366,...,0.018196,0.041177,0.039334,-0.044318,0.045463,0.025545,0.049202,0.000823,-0.038362,0.019681
2,-0.021999,-0.132681,-0.008762,-0.042748,-0.019589,-0.075033,-0.132914,0.005916,0.003201,0.049293,...,-0.015176,0.088349,0.074305,-0.060316,0.136529,0.038422,-0.004120,-0.041907,-0.089914,-0.001653
3,-0.058334,-0.032472,0.008695,-0.011285,-0.004759,-0.007067,-0.046322,0.025255,0.045424,0.050698,...,0.007130,0.038114,0.035892,-0.047891,0.040576,0.048087,0.053413,0.015304,-0.030066,0.023187
4,-0.029307,-0.066059,0.003879,-0.029705,-0.008513,-0.051460,-0.089139,-0.002922,0.013436,0.054816,...,0.013478,0.049914,0.044104,-0.035458,0.086094,0.047663,0.037057,-0.030735,-0.054160,0.024727


In [30]:
with open(f"emb_Polypharmacy_sample_{sample_no}.dat", "w") as file:
    file.write("\n")
    for idx, row in emb_df.iterrows():
        emb = row[:].astype(np.float32)
        emb_str = ' '.join(emb.astype(str))
        file.write(f'{idx}\t{emb_str}\n')
    